In [1]:
import pandas as pd
import glob
import csv
import os

# Read all files

In [2]:
cwd = os.getcwd()
cwd

'C:\\Users\\sebas\\Desktop\\spark'

In [3]:
#filenames=glob.glob('C:/Users/Eugene/Desktop/spark/Data_collected/*/part-[0-10]')
filenames = glob.glob('C:/Users/sebas/Desktop/spark/Data_collected/data*/part-*')
dfs = []

for filename in filenames:
    dfs.append(pd.read_csv(filename, sep='\t', header=None, names = ['Title','Description','Category']))

dfs

[                                               Title  \
 0  Trump Tariff Decision Looms, Setting Up a Glob...   
 
                                          Description  Category  
 0  European steel manufacturers were braced for n...  business  ,
                                                Title  \
 0  Growing Concern: Foreign Investors Lose Some H...   
 
                                          Description  Category  
 0  Foreign investors’ appetite this year for U.S....  business  ,
                                                Title  \
 0  Banks, Credit-Card Companies Explore Ways to M...   
 
                                          Description  Category  
 0  As financial firms have come under pressure fr...  business  ,
                                                Title  \
 0  Retail’s Other Problem: Too Few Clerks in the ...   
 
                                          Description  Category  
 0  Many of America’s biggest retailers, under ass...  business  ,
    

In [4]:
#Checkpoint
len(dfs) #want to have the number of columns for each column such that we can concataned the NumberColumns-1 and keep the last column as the class
len(dfs[1466])
dfs[1466]
dfs[1466]["Title"][0]
#title_vec[1:5]

'Instagram and WhatsApp are both getting group video calls soon'

In [5]:
#Extracting articles from each array in the list
title_vec=[]
desc_vec=[]
cat_vec=[]
for i in range(0,len(dfs)):
    title_vec.append(dfs[i]['Title'][0])
    desc_vec.append(dfs[i]["Description"][0])
    cat_vec.append(dfs[i]["Category"][0])
print(len(title_vec))
print(len(desc_vec))
print(len(cat_vec))

3679
3679
3679


In [6]:
#Creation of dataframe
art_df=pd.DataFrame({'Title': title_vec,
                      'Description': desc_vec,
                      'Category':cat_vec})
art_df[1:5]

,Category,Description,Title
1,business,Foreign investors’ appetite this year for U.S....,Growing Concern: Foreign Investors Lose Some H...
2,business,As financial firms have come under pressure fr...,"Banks, Credit-Card Companies Explore Ways to M..."
3,business,"Many of America’s biggest retailers, under ass...",Retail’s Other Problem: Too Few Clerks in the ...
4,business,Arizona parents scramble to find alternative a...,"When Teachers Strike, Parents Face Dilemma"


# Convert to CSV file

In [1]:
#Saving dataframe as CSV
#art_df.to_csv("C:/Users/Eugene/Desktop/spark/train_data.csv", sep=',')

In [8]:
#Checkpoint
art_df['Description'][0]

'European steel manufacturers were braced for new tariffs on their exports to America, as European Union efforts to avoid a trade war with the U.S. appeared to have failed.'

In [9]:
type(art_df.groupby(["Category"]).count().rename(columns={0:'count'}))

pandas.core.frame.DataFrame

In [4]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql import functions as sf

In [5]:
#Reading in dataset with category, title and description
#art_df=pd.read_csv("C:/Users/Eugene/Desktop/spark/train_data.csv",sep=',')
art_df2 = sqlContext.read.format("com.databricks.spark.csv").options(header=True, inferschema='true').load('C:/Users/Eugene/Desktop/spark/train_data.csv')

In [19]:
#SQLContext.createDataFrame(art_df)
art_df2.distinct().count()

3679

In [12]:
art_df2.show(5)
art_df2.columns

+---+--------+--------------------+--------------------+
|_c0|Category|         Description|               Title|
+---+--------+--------------------+--------------------+
|  0|business|European steel ma...|Trump Tariff Deci...|
|  1|business|Foreign investors...|Growing Concern: ...|
|  2|business|As financial firm...|Banks, Credit-Car...|
|  3|business|Many of America’s...|Retail’s Other Pr...|
|  4|business|Arizona parents s...|When Teachers Str...|
+---+--------+--------------------+--------------------+
only showing top 5 rows



['_c0', 'Category', 'Description', 'Title']

In [4]:
#Combing title and description together to obtain predictor words
art_df_new = art_df2.withColumn('Predictors',sf.concat(sf.col('Description'),sf.lit('_'), sf.col('Title')))
art_df_new = art_df_new.dropna(how="any")
art_df_new.show(5)

+---+--------+--------------------+--------------------+--------------------+
|_c0|Category|         Description|               Title|          Predictors|
+---+--------+--------------------+--------------------+--------------------+
|  0|business|European steel ma...|Trump Tariff Deci...|European steel ma...|
|  1|business|Foreign investors...|Growing Concern: ...|Foreign investors...|
|  2|business|As financial firm...|Banks, Credit-Car...|As financial firm...|
|  3|business|Many of America’s...|Retail’s Other Pr...|Many of America’s...|
|  4|business|Arizona parents s...|When Teachers Str...|Arizona parents s...|
+---+--------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [5]:
#Deleting Description, title and _c0 columns
art_df_new2=art_df_new.drop('Description', 'Title','_c0')
art_df_new2.show(5)

+--------+--------------------+
|Category|          Predictors|
+--------+--------------------+
|business|European steel ma...|
|business|Foreign investors...|
|business|As financial firm...|
|business|Many of America’s...|
|business|Arizona parents s...|
+--------+--------------------+
only showing top 5 rows



In [7]:
#Frequency of each category
from pyspark.sql.functions import col
art_df_new2.groupBy("Category").count().orderBy(col("count").desc()).show()

+-------------+-----+
|     Category|count|
+-------------+-----+
|     business|  773|
|       sports|  745|
|entertainment|  673|
|   technology|  524|
|      science|  359|
|       health|  275|
+-------------+-----+



# Pipeline for Text mining

In [6]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

In [7]:
# Transforming the texts into words
regexTokenizer = RegexTokenizer(inputCol="Predictors", outputCol="words", pattern="\\W")
# Removing stop words
#add_stopwords = ["http","https","amp","rt","t","c","the"] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")
# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

In [28]:
countVectors

CountVectorizer_4044a4c3d70d3e219949

In [22]:
art_df_new2.select(col('Category'))

DataFrame[Category: string]

In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

# Training

In [51]:
#Split data into test and train set
train, test = art_df_new2.randomSplit([0.8, 0.2])

In [10]:
from pyspark.ml.classification import LogisticRegression
# Create a LogisticRegression instance
lr = LogisticRegression(maxIter=10, regParam=0.01)
#lr2=LogisticRegressionModel()

In [54]:
#Transforms labels into numbers
cat_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, cat_stringIdx,lr])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(train) 

#Making predictions
data_pred = pipelineFit.transform(test) 
data_pred.show(5)

+--------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|Category|          Predictors|               words|            filtered|            features|label|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|business|"A cryptocurrenci...|[a, cryptocurrenc...|[cryptocurrencies...|(3227,[107,151,18...|  0.0|[1.96678790552497...|[0.66514463777804...|       0.0|
|business|"Arizona teachers...|[arizona, teacher...|[arizona, teacher...|(3227,[4,48,72,97...|  0.0|[4.22502344282267...|[0.96429996342497...|       0.0|
|business|"BI PRIME: Analys...|[bi, prime, analy...|[bi, prime, analy...|(3227,[65,210,424...|  0.0|[5.74130316926835...|[0.98906162540047...|       0.0|
|business|"BI PRIME: Analys...|[bi, prime, analy...|[bi, prime, analy...|(32

# Results (should change the way of presenting it)

In [56]:
#Show results, but don't figured out how to assess the performance... 
result = data_pred.select("features", "label", "rawPrediction", "prediction").collect()

for row in result:
    print("features=%s, label=%s -> prob=%s, prediction=%s"
          % (row.features, row.label, row.rawPrediction, row.prediction))

features=(3227,[107,151,186,322,476,548],[1.0,1.0,1.0,1.0,1.0,1.0]), label=0.0 -> prob=[1.9667879055249702,-0.06272803704440516,-0.21874111339404817,-0.07172966972900048,-0.5386133307583438,-1.0749757545991754], prediction=0.0
features=(3227,[4,48,72,97,374,457,468,807,1532,1980,2452],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]), label=0.0 -> prob=[4.2250234428226765,-0.9539210290073106,-0.13207585585427722,-0.24901024887075945,-1.1632582133252651,-1.7267580957650792], prediction=0.0
features=(3227,[65,210,424,552,592,1212,1278,1440,1511,1703,2191],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]), label=0.0 -> prob=[5.741303169268351,-1.5121935558644886,0.9747347737186047,-1.186136232169306,-1.9842372955907568,-2.033470859362403], prediction=0.0
features=(3227,[65,210,424,552,592,1212,1278,1440,1511,1703,2191],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]), label=0.0 -> prob=[5.741303169268351,-1.5121935558644886,0.9747347737186047,-1.186136232169306,-1.9842372955907568,-2.033470859362

features=(3227,[22,36,64,275,315,405,451,526,628,664,747,998,1062,1240,1339,1899,2164,2405,2512,3063],[2.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]), label=3.0 -> prob=[-0.7175387415344026,-2.2725350662393344,-0.30060799728199866,5.872785427081512,-0.1509724018862758,-2.4311312201394957], prediction=3.0
features=(3227,[8,696,1620,1962,2005,2028],[2.0,2.0,1.0,2.0,1.0,2.0]), label=3.0 -> prob=[-0.3575581755816375,-0.787960480204196,-0.4894674214662243,4.923157356176877,-1.6432871029303089,-1.6448841759945236], prediction=3.0
features=(3227,[6,81,87,134,216,266,2446],[1.0,1.0,1.0,1.0,1.0,1.0,1.0]), label=3.0 -> prob=[0.3264415933943943,0.29351300239970174,0.3341797150514719,0.8686942852473832,-0.7422841399810616,-1.080544456111894], prediction=3.0
features=(3227,[6,81,87,134,216,266,2446],[1.0,1.0,1.0,1.0,1.0,1.0,1.0]), label=3.0 -> prob=[0.3264415933943943,0.29351300239970174,0.3341797150514719,0.8686942852473832,-0.7422841399810616,-1.080544456111894]

In [58]:
#To see prediction per label
data_pred.filter(data_pred['prediction'] == 2).select("Predictors","Category","probability","label","prediction").orderBy("probability", ascending=False).show(n = 10, truncate = 30)


+------------------------------+-------------+------------------------------+-----+----------+
|                    Predictors|     Category|                   probability|label|prediction|
+------------------------------+-------------+------------------------------+-----+----------+
|Realizing that embracing Co...|     business|[0.3675162862431996,0.02981...|  0.0|       2.0|
|Sandeep Sharma, Siddarth Ka...|       sports|[0.30614213163779574,0.1239...|  1.0|       2.0|
|The comedian broke his sile...|entertainment|[0.287420741858188,0.017797...|  2.0|       2.0|
|A plant fossil that lay unn...|      science|[0.2324820327531092,0.13129...|  4.0|       2.0|
|The parent company of KFC, ...|     business|[0.22574039619636116,0.0131...|  0.0|       2.0|
|Meanwhile, Canada looks to ...|     business|[0.22300046332576018,0.0185...|  0.0|       2.0|
|The cast and crew have come...|entertainment|[0.2218780998890314,0.18882...|  2.0|       2.0|
|"Hint: Four syllables, rhym...|   technology|[0.1

In [59]:
#predictions.select("prediction").show(1)
data_pred.select("prediction").toPandas().prediction[50]

0.0

# Model Evaluation

In [ ]:
#Accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(data_pred)
print("Test Error = %g " % (1.0 - accuracy))
print("Accuracy = %g" % accuracy)

# Online predictions

In [14]:
from threading import Thread
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession

In [16]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [62]:
#Thread
class StreamingThread(Thread):
    def __init__(self, ssc):
        Thread.__init__(self)
        self.ssc = ssc
    def run(self):
        ssc.start()
        ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)


def getSparkSessionInstance(sparkConf):
    if ("sparkSessionSingletonInstance" not in globals()):
        globals()["sparkSessionSingletonInstance"] = SparkSession \
            .builder \
            .config(conf=sparkConf) \
            .getOrCreate()
    return globals()["sparkSessionSingletonInstance"]




#def predict(TitDesc):
 #   rdd_titdesc = sc.parallelize([(TitDesc,'NotARealCategory')])
#    TitDesc_rdd=rdd_titdesc.map(lambda title:Row(Category=title[1],Predictors=title[0]))
 #   TitDesc_df=spark.createDataFrame(TitDesc_rdd)
  #  pipelineFit = globals()['pipeline'].fit(TitDesc_df)
   # TitDesc_p = globals()['pipelineFit'].transform(TitDesc_df)
    #pred_df= globals()['loadedModel'].transform(TitDesc_df)
    #pred=predictions.select("prediction").toPandas().prediction[0]
     
    return pred

globals()['model1']=False
def process(time, rdd):
    if rdd.isEmpty():
        return
    print("========= %s =========" % str(time))   
    
    rowRdd = rdd.map(lambda line: Row(Predictors=line.split('\t')[0]+line.split('\t')[1],
        Category=line.split('\t')[2]))
        
    df_strm = spark.createDataFrame(rowRdd)
    df_result = pipelineFit.transform(df_strm).select("Predictors","Category","prediction")
    df_result.show()
    #df.show()

In [64]:
ssc = StreamingContext(sc, 10)

In [65]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

LABELS: Business=0.0, Sports=1.0, Entertainment=2.0, Science=4.0, Technology=3.0, 

In [66]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2018-06-02 22:04:10 =========
+--------------------+--------+----------+
|          Predictors|Category|prediction|
+--------------------+--------+----------+
|G7 Members Issue ...|business|       0.0|
|Remaining French ...|business|       0.0|
+--------------------+--------+----------+



In [67]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
========= 2018-06-02 22:04:20 =========
+--------------------+--------+----------+
|          Predictors|Category|prediction|
+--------------------+--------+----------+
|Jim Mattis Warns ...|business|       0.0|
|Former Houston Te...|business|       1.0|
|Trump's legal tea...|business|       0.0|
|The Overprotected...|business|       2.0|
|Can “effective al...|business|       3.0|
|Turkey’s oppositi...|business|       0.0|
|Ben Affleck's Pri...|business|       0.0|
+--------------------+--------+----------+

========= 2018-06-02 22:04:30 =========
+--------------------+--------+----------+
|          Predictors|Category|prediction|
+--------------------+--------+----------+
|Lunch with Warren...|business|       3.0|
|Blockchain start-...|business|       0.0|
|FDA chief outline...|business|       0.0|
|I'm addicted to m...|business|       2.0|
|How to avoid a su...|business|       2.0|
|Creep Sweep: Spy ...|business|       0.0|
|What t

# Hyperparameter tuning 

In [67]:
"Marathon Petroleum to Buy Andeavor for More Than $23 BillionMarathon Petroleum to Buy Andeavor for More Than $23 Billion Marathon Petroleum agreed to buy pipeline and refining company Andeavor for $23.3 billion, the latest deal in an active year for energy-related mergers. business"

'Marathon Petroleum to Buy Andeavor for More Than $23 BillionMarathon Petroleum to Buy Andeavor for More Than $23 Billion Marathon Petroleum agreed to buy pipeline and refining company Andeavor for $23.3 billion, the latest deal in an active year for energy-related mergers. business'

In [2]:
# https://spark.apache.org/docs/2.3.0/ml-tuning.html + See what group last year did

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [2]:
# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="Predictors", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
cat_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")
lr = LogisticRegression(maxIter=10)
pipeline = Pipeline(stages=[cat_stringIdx,tokenizer, hashingTF, lr])

In [47]:
# Pipeline treated as an Estimator, wrapping it in a CrossValidator instance.
# This will allow us to jointly choose parameters for all Pipeline stages.
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# With 3 values for hashingTF.numFeatures and 2 values for lr.regParam,
# this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.

paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator= MulticlassClassificationEvaluator(),
                          numFolds=2)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(train)

In [1]:
#Predictions
cvModel.transform(test).show(5)